# Prepare ARC Trip Data
Retrieved from: https://abmfiles.atlantaregional.com/downloads (downloaded Full Model Run - RTP Amd 6 - 2030)

TODO re-download this and see if I can host it?

In [ ]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

from bikewaysim.paths import config
from bikewaysim.framework import bikewaysim_functions
from bikewaysim.routing import rustworkx_routing_funcs

In [ ]:
taz_fp = config['raw_data_fp'] / "Atlanta_Regional_Commission/Model_Traffic_Analysis_Zones_2020.geojson"
trip_fp = list((config['raw_data_fp'] / "Atlanta_Regional_Commission/rtp_amd6_2030_tripData").glob("*"))
outputs = []

In [3]:
links, turns, length_dict, geo_dict, turn_G = rustworkx_routing_funcs.import_calibration_network(config)
nodes = gpd.read_file(config['network_fp'] / 'networks.gpkg',layer='osm_nodes')
nodes.rename(columns={'osm_N':'N'},inplace=True)
# nodes = gpd.read_file(config['network_fp']/"final_network.gpkg",layer="nodes") #BUG why is this only 4k nodes?
node_ids = set(links['A'].tolist() + links['B'].tolist())
nodes = nodes[nodes['N'].isin(node_ids)]

# Initial Count

In [ ]:
# to get how many tazs
all_tazs = gpd.read_file(taz_fp,ignore_geometry=True).shape[0]

### Import ARC Trip Data from TIP Amendment Six 2030 Model Run
### https://abmfiles.atlantaregional.com/downloads/zPrevious%20Model%20-%20Loaded%20Highway%20Networks%20-%20GIS%20-%20RTP%20Amd%206%20-%202030.zip
tripData0 = pd.read_csv(trip_fp[0],usecols=['orig_taz','dest_taz','distance','hh_id','person_id'])
tripData1 = pd.read_csv(trip_fp[1],usecols=['orig_taz','dest_taz','distance','hh_id','person_id'])
tripData2 = pd.read_csv(trip_fp[2],usecols=['orig_taz','dest_taz','distance','hh_id','person_id'])
tripData = pd.concat([tripData0,tripData1,tripData2])
del tripData0, tripData1, tripData2

tripData['hh_id'] = tripData['hh_id'].astype(str)
tripData['person_id'] = tripData['person_id'].astype(str)
tripData['orig_taz'] = tripData['orig_taz'].astype(str)
tripData['dest_taz'] = tripData['dest_taz'].astype(str)

outputs.append([
    all_tazs,
    tripData.groupby(['orig_taz','dest_taz']).ngroups,
    tripData.shape[0],
    tripData['hh_id'].nunique(),
    tripData['person_id'].nunique()
])

# Within Study Area

In [ ]:
studyarea = gpd.read_file(config['studyarea_fp'])
tazs = gpd.read_file(taz_fp,mask=studyarea)[['MTAZ10','geometry']]
tazs.rename(columns={'MTAZ10':'tazid'},inplace=True)
tazs['tazid'] = tazs['tazid'].astype(int).astype(str)

tripData = tripData[tripData['orig_taz'].isin(tazs['tazid']) & tripData['dest_taz'].isin(tazs['tazid'])]

outputs.append([
    tazs.shape[0],
    tripData.groupby(['orig_taz','dest_taz']).ngroups,
    tripData.shape[0],
    tripData['hh_id'].nunique(),
    tripData['person_id'].nunique()
])

# Remove if matching distance exceeded 500 feet
Euclidean matching distance is less than or equal to 500 feet (~2.5 minutes walking at 2 mph)

In [ ]:
## Snap TAZs to network

tazs = bikewaysim_functions.snap_ods_to_network(tazs,nodes)

#filter
tazs = tazs[tazs['dist']<=500]
tripData = tripData[tripData['orig_taz'].isin(tazs['tazid']) & tripData['dest_taz'].isin(tazs['tazid'])]

outputs.append([
    tazs.shape[0],
    tripData.groupby(['orig_taz','dest_taz']).ngroups,
    tripData.shape[0],
    tripData['hh_id'].nunique(),
    tripData['person_id'].nunique()
])

# Remove Trips Above 3 Miles


In [ ]:
tripData = tripData[tripData['distance'] <= 3]

#filter tazs
tazs = tazs[tazs['tazid'].isin(tripData['orig_taz']) | tazs['tazid'].isin(tripData['dest_taz'])]

outputs.append([
    tazs.shape[0],
    tripData.groupby(['orig_taz','dest_taz']).ngroups,
    tripData.shape[0],
    tripData['hh_id'].nunique(),
    tripData['person_id'].nunique()
])

# Remove Intrazonal Trips


In [ ]:
tripData = tripData[tripData['orig_taz'] != tripData['dest_taz']]

#filter tazs
tazs = tazs[tazs['tazid'].isin(tripData['orig_taz']) | tazs['tazid'].isin(tripData['dest_taz'])]

outputs.append([
    tazs.shape[0],
    tripData.groupby(['orig_taz','dest_taz']).ngroups,
    tripData.shape[0],
    tripData['hh_id'].nunique(),
    tripData['person_id'].nunique()
])

In [ ]:
#condense tripData
tripData = tripData.groupby(['orig_taz','dest_taz']).size().reset_index()
tripData.columns = ['orig_taz','dest_taz','trips']

# Final Count

In [ ]:
outputs_df = pd.DataFrame(
    outputs,
    columns=['Number of TAZs','Number of OD Pairs','Number of Trips','Number of Households','Number of Persons'],
    index=['Initial Count','Within Study Area','Match Distance Filter','Trip Distance Filter','Remove Intrazonal']
    )
outputs_df.to_csv(config['scratch_fp']/'taz_stats.csv',index=True)
outputs_df.head()

# Add snapped nodes to trip data


In [ ]:
snapped_ods = dict(zip(tazs['tazid'],tazs['N']))
tripData['orig_N'] = tripData['orig_taz'].map(snapped_ods)
tripData['dest_N'] = tripData['dest_taz'].map(snapped_ods)

Create trips per origin/destination visual

In [ ]:
# trip_origins_per_taz = tripData.groupby('orig_taz')['trips'].sum().reset_index()
# trip_origins_per_taz.columns = ['tazid','trip_origins']
# trip_destinations_per_taz = tripData.groupby('dest_taz')['trips'].sum().reset_index()
# trip_destinations_per_taz.columns = ['tazid','trip_destinations']

# trips_per_taz = tazs.copy()
# trips_per_taz = trips_per_taz.merge(trip_origins_per_taz,on='tazid',how='left')
# trips_per_taz = trips_per_taz.merge(trip_destinations_per_taz,on='tazid',how='left')
# trips_per_taz['both'] = trips_per_taz['trip_origins'] + trips_per_taz['trip_destinations']
# trips_per_taz.fillna(0,inplace=True)
# ax = trips_per_taz.plot('both',cmap='YlGnBu',edgecolor='grey')
# plt.axis('off')
# plt.show()

In [ ]:
trip_origins_per_taz = tripData.groupby('orig_taz')['trips'].sum().reset_index()
trip_origins_per_taz.columns = ['tazid','trip_origins']
trip_origins_per_taz = tazs.merge(trip_origins_per_taz,on='tazid')
trip_origins_per_taz.plot('trip_origins')
trip_origins_per_taz.to_file(config['bikewaysim_fp']/'ods.gpkg',layer='trip_origins')

In [ ]:
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import MultipleLocator

def thousands(x, pos):
    'The two args are the value and tick position'
    return '%1.1f' % (x * 1e-3)
formatter = FuncFormatter(thousands)

fig, ax = plt.subplots()

bins = range(0, 50000, 1000)
ax.hist(trip_origins_per_taz['trip_origins'], bins=bins, color='grey',alpha=0.3,edgecolor='black')
ax.set_xlabel(f"Trip Origins Per TAZ in Thousands (N = {trip_origins_per_taz['trip_origins'].sum() / 10**6 :.1f} Million)")
ax.set_ylabel(f"Number of TAZs (N = {trip_origins_per_taz.shape[0]:,})")
ax.axvline(trip_origins_per_taz['trip_origins'].mean(), color='red', linestyle='dashed', linewidth=1)
ax.text(trip_origins_per_taz['trip_origins'].mean() * 1.5, plt.gca().get_ylim()[1] * 0.9, f"Mean: {trip_origins_per_taz['trip_origins'].mean():,.0f} origins", color='red', ha='left')

ax.xaxis.set_major_locator(MultipleLocator(10**4))
ax.xaxis.set_minor_locator(MultipleLocator(10**3))
ax.yaxis.set_major_locator(MultipleLocator(50))
ax.yaxis.set_minor_locator(MultipleLocator(25))
ax.xaxis.set_major_formatter(formatter)

plt.savefig(config['figures_fp']/'number_of_trip_origins_hist.png',dpi=300)
plt.show()

In [ ]:
trip_origins_per_taz['trip_origins'].mean()

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt

# # Example DataFrame
# trip_origins_per_taz = pd.DataFrame({'trip_origins': [10000, 20000, 15000, 5000, 25000]})

# # Set the font to Times New Roman
# plt.rcParams['font.family'] = 'Times New Roman'

# # Create histogram
# plt.hist(trip_origins_per_taz['trip_origins'], bins=range(0, 50000, 2000), color='grey')

# # Remove grid lines
# plt.grid(False)

# # Calculate the sum of trip origins with comma formatting
# total_trip_origins = f"{trip_origins_per_taz['trip_origins'].sum():,}"

# # Add axis labels with formatted number
# plt.xlabel(f"Trip Origins Per TAZ (N = {total_trip_origins})")
# plt.ylabel('Frequency')

# # Calculate the mean value
# mean_value = trip_origins_per_taz['trip_origins'].mean()

# # Add vertical red line at the mean value
# plt.axvline(mean_value, color='red', linestyle='dashed', linewidth=1)

# # Add label for the mean value
# plt.text(mean_value, plt.gca().get_ylim()[1] * 0.9, f'Mean: {mean_value:,.0f}', color='red', ha='center')

# # Show plot
# plt.show()


In [ ]:
# trip_destinations_per_taz = tripData.groupby('dest_taz')['trips'].sum().reset_index()
# trip_destinations_per_taz.columns = ['tazid','trip_destinations']
# trip_destinations_per_taz = tazs.merge(trip_destinations_per_taz,on='tazid')
# trip_destinations_per_taz.plot('trip_destinations')

## Get the TAZs within the study area that were filtered out or had zero trips

In [ ]:
studyarea = gpd.read_file(config['studyarea_fp'])
tazs0 = gpd.read_file(taz_fp,mask=studyarea)[['MTAZ10','geometry']]
tazs0.rename(columns={'MTAZ10':'tazid'},inplace=True)
tazs0['tazid'] = tazs0['tazid'].astype(int).astype(str)

tazs0 = tazs0[tazs0['tazid'].isin(tazs['tazid'])==False]
tazs0.plot()

tazs0.to_file(config['bikewaysim_fp']/'ods.gpkg',layer='filtered_out_or_zero')

In [ ]:
# # trip origins per TAZ

# #export the taz areas
# tazs.to_crs(config['projected_crs_epsg']).to_file(config['bikewaysim_fp']/'map_layers.gpkg',layer='taz_polygons')
# #export the taz centroids
# taz_centroids = tazs.copy().to_crs(config['projected_crs_epsg'])
# taz_centroids.geometry = taz_centroids.geometry.centroid
# taz_centroids.to_crs(config['projected_crs_epsg']).to_file(config['bikewaysim_fp']/'map_layers.gpkg',layer='taz_centroids')

In [ ]:
#export
tripData.to_csv(config['bikewaysim_fp']/'od_matrix.csv',index=False)

## Exploration Past Here

In [ ]:
# tripData0 = pd.read_csv(trip_fp[0])
# tripData0[['hh_id','person_id']].nunique()
# tripData0['trip_mode'].value_counts()
# tripData0.groupby('hh_id')['person_id'].nunique().value_counts()
# tripData0[tripData0['person_num']>4].sort_values('hh_id').head(20)
# tripData0.groupby('person_id')['tour_id'].nunique().value_counts()
# tripData0.columns
# # tazs.to_file(config['bikewaysim_fp']/'ods.gpkg',layer='tazs')